In [1]:
# 📌 Import Required Libraries
import os
from dotenv import load_dotenv
from typing_extensions import TypedDict
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, START, END

# ✅ Load Environment Variables
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

# ✅ Initialize AI Model (Qwen-2.5-32B)
llm = ChatGroq(model="qwen-2.5-32b")

# ✅ Define AI State (Data Stored in Workflow)
class State(TypedDict):
    topic: str
    joke: str
    improved_joke: str
    final_joke: str

# ✅ Step 1: Generate Initial Joke
def generate_joke(state: State):
    """First LLM call to generate an initial joke."""
    msg = llm.invoke(f"Write a short joke about {state['topic']}")
    return {"joke": msg.content}

# ✅ Step 2: Check if the Joke Has a Punchline
def check_punchline(state: State):
    """Gate function to check if the joke has a punchline."""
    if "?" in state["joke"] or "!" in state["joke"]:
        return "Fail"  # Needs improvement
    return "Pass"  # Joke is good as-is

# ✅ Step 3: Improve the Joke if Needed
def improve_joke(state: State):
    """Second LLM call to improve the joke."""
    msg = llm.invoke(f"Make this joke funnier by adding wordplay: {state['joke']}")
    return {"improved_joke": msg.content}

# ✅ Step 4: Polish the Joke with a Surprising Twist
def polish_joke(state: State):
    """Third LLM call for final polish."""
    msg = llm.invoke(f"Add a surprising twist to this joke: {state['improved_joke']}")
    return {"final_joke": msg.content}

# ✅ Build LangGraph Workflow
workflow = StateGraph(State)

# ➤ Add Nodes
workflow.add_node("generate_joke", generate_joke)
workflow.add_node("improve_joke", improve_joke)
workflow.add_node("polish_joke", polish_joke)

# ➤ Define Execution Flow
workflow.add_edge(START, "generate_joke")  # Start → Generate Joke
workflow.add_conditional_edges(
    "generate_joke", check_punchline, 
    {"Fail": "improve_joke", "Pass": END}  # If joke fails, improve it
)
workflow.add_edge("improve_joke", "polish_joke")  # Improve → Polish
workflow.add_edge("polish_joke", END)  # Finish after polish

# ✅ Compile the Workflow
chain = workflow.compile()

# ✅ Run AI Joke Generator
def generate_joke_for_topic(topic: str):
    """
    Runs the AI joke generator for a given topic.
    """
    state = chain.invoke({"topic": topic})
    
    if "final_joke" in state:
        print("\n🎭 Final AI-Generated Joke:\n", state["final_joke"])
    elif "joke" in state:
        print("\n🎭 AI-Generated Joke:\n", state["joke"])

# ✅ Example Run
if __name__ == "__main__":
    topic = input("Enter a joke topic: ")
    generate_joke_for_topic(topic)



🎭 Final AI-Generated Joke:
 Why don't cats play poker in the wild? Because every game ends up with a cheetah calling their bluff—and it turns out, the cheetah is actually a card shark in a big cat disguise, running circles around the other animals in more ways than one!
